In [ ]:
def make_model(settings, x_train, onehot_train, model_compile=True):
    model = build_shash_model(
        x_train,
        onehot_train,
        hiddens=settings["hiddens"],
        output_shape=onehot_train.shape[1],
        ridge_penalty=settings["ridge_param"],
        act_fun=settings["act_fun"],
        dropout_rate=settings["dropout_rate"],
        rng_seed=settings["rng_seed"],                    
    )

    if model_compile == True:        
        model.compile(
            optimizer=optimizers.Adam(
                learning_rate=settings["learning_rate"]
            ),
            loss=compute_shash_NLL,
            metrics=[
                CustomMAE(name="custom_mae"),
                InterquartileCapture(name="interquartile_capture"),
                SignTest(name="sign_test"),
            ],
        )
    
    return model
    
def build_shash_model(
    x_train, onehot_train, hiddens, output_shape, ridge_penalty=[0.0,], act_fun="relu", rng_seed=999, dropout_rate=[0.0,],
):
    """Build the fully-connected shash network architecture with
    internal scaling.

    Arguments
    ---------
    x_train : numpy.ndarray
        The training split of the x data.
        shape = [n_train, n_features].

    onehot_train : numpy.ndarray
        The training split of the scaled y data is in the first column.
        The remaining columns are filled with zeros. The number of columns
        equal the number of distribution parameters.
        shape = [n_train, n_parameters].

    hiddens : list (integers)
        Numeric list containing the number of neurons for each layer.

    output_shape : integer {2, 3, 4}
        The number of distribution output parameters to be learned.

    ridge_penalty : float, default=0.0
        The L2 regularization penalty for the first layer.

    act_fun : function, default="relu"
        The activation function to use on the deep hidden layers.

    Returns
    -------
    model : tensorflow.keras.models.Model

    Notes
    -----
    * The number of output units is determined by the output_shape argument.
        If output_shape is:
        2 -> output_layer = [mu_unit, sigma_unit]
        3 -> output_layer = [mu_unit, sigma_unit, gamma_unit]
        4 -> output_layer = [mu_unit, sigma_unit, gamma_unit, tau_unit]

    * Unlike most of EAB's models, the features are normalized within the
        network.  That is, the x_train, y_train, ... y_test should not be
        externally normalized or scaled.

    * In essence, the network is learning the shash parameters for the
        normalized y values. Say mu_z and sigma_z where

            z = (y - y_avg)/y_std

        The mu_unit and sigma_unit layers rescale the learned mu_z
        and sigma_z parameters back to the dimensions of the y values.
        Specifically,

            mu_y    = y_std * mu_z + y_avg
            sigma_y = y_std * sigma_z

        However, since the model works with log(sigma) we must use

            log(sigma_y) = log(y_std * sigma_z) = log(y_std) + log(sigma_z)

    * Note the gamma and tau parameters of the shash distribution are
        dimensionless by definition. So we do not need to rescale gamma
        and tau.

    """
    # set inputs
    if len(hiddens) != len(ridge_penalty):
        ridge_penalty = np.ones(np.shape(hiddens))*ridge_penalty
    if len(hiddens) != len(dropout_rate):
        dropout_rate = np.ones((len(hiddens)+1,))*dropout_rate
    
    # The avg and std for feature normalization are computed from x_train.
    # Using the .adapt method, these are set once and do not change, but
    # the constants travel with the model.
    inputs = tf.keras.Input(shape=x_train.shape[1:])
    
    x = tf.keras.layers.Dropout(
        rate=dropout_rate[0],
        seed=rng_seed,            
    )(inputs)        

    # linear network only
    if hiddens[0] == 0:
        x = tf.keras.layers.Dense(
            units=1,
            activation="linear",
            use_bias=True,
            kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=ridge_penalty[0]),
            bias_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+0),
            kernel_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+0),
        )(x)
    else:
        # Initialize the first hidden layer.
        x = tf.keras.layers.Dense(
            units=hiddens[0],
            activation=act_fun,
            use_bias=True,
            kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=ridge_penalty[0]),
            bias_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+0),
            kernel_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+0),
        )(x)

        # Initialize the subsequent hidden layers.
        for ilayer, layer_size in enumerate(hiddens[1:]):
            
            x = tf.keras.layers.Dropout(
                rate=dropout_rate[ilayer+1],
                seed=rng_seed,            
            )(x)            
            
            x = tf.keras.layers.Dense(
                units=layer_size,
                activation=act_fun,
                use_bias=True,
                kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=ridge_penalty[ilayer+1]),
                bias_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+ilayer+1),
                kernel_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+ilayer+1),
            )(x)
            
    # Compute the mean and standard deviation of the y_train data to rescale
    # the mu and sigma parameters.
    y_avg = np.mean(onehot_train[:, 0])
    y_std = np.std(onehot_train[:, 0])

    # mu_unit.  The network predicts the scaled mu_z, then the resclaing
    # layer scales it up to mu_y.
    mu_z_unit = tf.keras.layers.Dense(
        units=1,
        activation="linear",
        use_bias=True,
        bias_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+100),
        kernel_initializer=tf.keras.initializers.RandomNormal(seed=rng_seed+100),
        name="mu_z_unit",
    )(x)

    mu_unit = tf.keras.layers.Rescaling(
        scale=y_std,
        offset=y_avg,
        name="mu_unit",
    )(mu_z_unit)

    # sigma_unit. The network predicts the log of the scaled sigma_z, then
    # the resclaing layer scales it up to log of sigma y, and the custom
    # Exponentiate layer converts it to sigma_y.
    log_sigma_z_unit = tf.keras.layers.Dense(
        units=1,
        activation="linear",
        use_bias=True,
        bias_initializer=tf.keras.initializers.Zeros(),
        kernel_initializer=tf.keras.initializers.Zeros(),
        name="log_sigma_z_unit",
    )(x)

    log_sigma_unit = tf.keras.layers.Rescaling(
        scale=1.0,
        offset=np.log(y_std),
        name="log_sigma_unit",
    )(log_sigma_z_unit)

    sigma_unit = Exponentiate(
        name="sigma_unit",
    )(log_sigma_unit)

    # gamma_unit. The network predicts the gamma directly.
    gamma_unit = tf.keras.layers.Dense(
        units=1,
        activation="linear",
        use_bias=True,
        bias_initializer=tf.keras.initializers.Zeros(),
        kernel_initializer=tf.keras.initializers.Zeros(),
        name="gamma_unit",
    )(x)

    # tau_unit. The network predicts the log of the tau, then
    # the custom Exponentiate layer converts it to tau.
    log_tau_unit = tf.keras.layers.Dense(
        units=1,
        activation="linear",
        use_bias=True,
        bias_initializer=tf.keras.initializers.Zeros(),
        kernel_initializer=tf.keras.initializers.Zeros(),
        name="log_tau_unit",
    )(x)

    tau_unit = Exponentiate(
        name="tau_unit",
    )(log_tau_unit)

    output_layer = tf.keras.layers.concatenate(
        [mu_unit, sigma_unit, gamma_unit, tau_unit], axis=1
    )

    model = tf.keras.models.Model(inputs=inputs, outputs=output_layer)
    return model